In [17]:
from bs4 import BeautifulSoup
import time
import requests
from random import randint
from html.parser import HTMLParser
from urllib.parse import urlparse, unquote
import json
import subprocess
import re
import pandas as pd

In [49]:
USER_AGENT = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0 Win64x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

class SearchEngine:
    @staticmethod
    def search(query, sleep=True):
        if sleep:  # Prevents loading too many pages too soon
            time.sleep(randint(10, 100))
        search_results = []
        url = 'http://www.search.yahoo.com/search?p=' + '+'.join(query.split()) + '&n=30'
        while(len(search_results)<10):
            soup = BeautifulSoup(requests.get(url, headers=USER_AGENT).text,'html.parser')
            scrape_results=SearchEngine.scrape_search_result(soup)
            search_results.extend(x for x in scrape_results if x not in search_results)
            url = soup.find("a", attrs = {"class" : "next"})['href']
            time.sleep(100)
            
        return search_results[:10]   

    @staticmethod
    def scrape_search_result(soup):
        raw_results = soup.find_all("a", attrs = {"class" : "d-ib fz-20 lh-26 td-hu tc va-bot mxw-100p"})
        results = []
    # implement a check to get only 10 results and also check that URLs must not be duplicated
        for result in raw_results:
            link = unquote(urlparse(result['href']).path.split('/')[5].split('=')[1])
            results.append(link)
        return results



In [51]:
yahoo = {}
n=0
with open("./Data/100QueriesSet2.txt") as file:
    for line in file:
        n+=1
        print(n,"-->",line.rstrip())
        #turn off/on the wifi every 3rd query in order to prevent from IPaddress getting banned by the Yahoo browser.
        if(n%3==0):
            wifi_off = subprocess.Popen(["networksetup", "-setairportpower", "airport", "off"], stdout=subprocess.PIPE)
            time.sleep(10)
            wifi_on = subprocess.Popen(["networksetup", "-setairportpower", "airport", "on"], stdout=subprocess.PIPE)
            time.sleep(100)
        search_url_list = SearchEngine.search(line.rstrip())
        if len(search_url_list) == 0:
            break
        else:
            yahoo[line.rstrip()] = search_url_list

1 --> Some important facts on the respiratory system
2 --> You want to text dirty with your boyfriend what should you say
3 --> Can you give me example of anecdotes by the filipino writers
4 --> How do you figure sq ft for a house
5 --> What city is the iffel tower
6 --> Hw big are sperm whales up to dinosaurs
7 --> What is not part of a biology
8 --> What is the radio code for a 2005 renault clio sport 182 cup
9 --> How many troops are in a modern marine battalion
10 --> More defines for respect
11 --> Why your deserts so hot
12 --> Are the decomposer that live in the ocean
13 --> Site where vladimir lenin is kept
14 --> Codes to get weapons in poptropica
15 --> How many grammys does alicia keys have now
16 --> What level does pineco evovle at
17 --> How many toes does an ostrich have
18 --> How does an iphone compare to a blackberry
19 --> How did darius reorganize and improve the persian government
20 --> What are lyrics of the star spangled banner
21 --> What do the uniting church 

In [52]:
with open("hw1.json", "w") as outfile:
    outfile.write(json.dumps(yahoo, indent=4))

In [53]:
def regex_url(url_list):
    for i in range(len(url_list)):
        url_list[i] = re.sub("https?://(www.)?", "", url_list[i]).rstrip('/')
    return url_list

def find_rho(rank_table,overlaps):
    if overlaps == 0:
        rho = 0
    elif overlaps == 1:
        if rank_table[0][0] == rank_table[0][1]:
            rho = 1
        else:
            rho = 0
    else:
        summation = sum([(g-y)**2 for g,y in rank_table])
        rho = 1 - ((6 * summation) / (overlaps * (overlaps ** 2 - 1)))
    return rho

def find_overlaps(yahoo_query,google_query,n):
    overlaps = 0
    rank_table = []
    for yahoo_idx in range(len(yahoo_query)):
        for google_idx in range(len(google_query)):
            if yahoo_query[yahoo_idx] == google_query[google_idx]:
                overlaps+=1
                rank_table.append([google_idx+1,yahoo_idx+1])
    rho = find_rho(rank_table,overlaps)
    return ['Query '+str(n),overlaps,(overlaps/len(google_query))*100,rho]

In [54]:
yahoo_ = json.load(open('hw1.json'))
google = json.load(open('./Data/Google_Result2.json'))
n=0
csv = []
with open("./Data/100QueriesSet2.txt") as file:
    for line in file:
        n+=1
        yahoo_query = regex_url(yahoo_[line.rstrip()])
        google_query = regex_url(google[line.rstrip()])
        csv.append(find_overlaps(yahoo_query,google_query,n))
csv_pd = pd.DataFrame(csv,columns =['Queries', 'Number of overlapping results', 'Percent Overlap', 'Spearman Coeffiient'])
averages = csv_pd[['Number of overlapping results', 'Percent Overlap', 'Spearman Coeffiient']].mean()
csv_pd.loc[len(csv_pd)] = ['Averages',averages[0],averages[1],averages[2]]
csv_pd.to_csv('hw1.csv', index=False)

#### References
[1] https://pytutorial.com/python-remove-https-http-from-url/

[2] https://docs.python.org/3/library/urllib.parse.html

[3] https://docs.python.org/2/library/urllib2.html

[4] https://stackoverflow.com/questions/5262470/function-in-python-to-clean-up-and-normalize-a-url

[5] https://stackoverflow.com/questions/37313958/turn-on-off-wifi-with-python-on-osx

[6] https://stackoverflow.com/questions/26934268/web-scraping-yahoo-provides-dirtyurl-instead-of-normal-url

[7] https://stackoverflow.com/questions/33940917/how-can-i-remove-the-last-character-of-a-string-in-python